In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.chdir("drive/MyDrive/Colab Notebooks/Authorship Identification/")

import torch
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import DistilBertTokenizer, BertTokenizer, DebertaTokenizer

random_seed = 42
np.random.seed = 42
torch.random.seed = 42

/home/ys724/.conda/envs/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ys724/.conda/envs/venv/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Load C50 Dataset (if already loaded)

- this (original dataset) is 50 train - 50 test split

In [23]:
train_df = pd.read_csv('C50/train.csv')
test_df = pd.read_csv('C50/test.csv')

- this dataset is the original one with entity names removed

In [2]:
train_df = pd.read_csv('C50/train_noents.csv')
test_df = pd.read_csv('C50/test_noents.csv')
train_df.rename(columns={'text': 'content', 'label': 'author'}, inplace=True)
test_df.rename(columns={'text': 'content', 'label': 'author'}, inplace=True)

- this is 90 train - 10 test split

In [17]:
test_frac = 0.1
train_df = pd.concat([train_df, test_df])
test_df = train_df.sample(frac=test_frac, random_state=random_seed)
train_df = train_df.drop(test_df.index)

In [3]:
train_df.head()

,content,author
0,"The capital market, roundly criticised for a l...",1
1,beat the Republic 3-0 in a ice hockey game on ...,1
2,The fall in the trade deficit to crowns in fro...,1
3,paper concern a.s. on said its net profit fell...,1
4,shares rallied on following the coalition gove...,1


### Load "All the News" 1 Dataset

In [2]:
train_df = pd.read_csv('All the News/all_the_news1_noents.csv')
train_df = train_df[(train_df.publication == 'Breitbart')]
top10_authors = train_df[['author','id']].groupby('author').count().sort_values(by='id', ascending=False).head(10).reset_index() # this == what the paper proposed
print('The authors are', top10_authors.author.values)
train_df = pd.merge(train_df, top10_authors[['author']], on='author', how='right')
train_df = train_df.groupby(by='author').sample(500, random_state=random_seed)
train_df.shape

The authors are ['Breitbart News' 'Pam Key' 'Charlie Spiering' 'Jerome Hudson'
 'John Hayward' 'Daniel Nussbaum' 'AWR Hawkins' 'Ian Hanchett'
 'Joel B. Pollak' 'Alex Swoyer']


(5000, 10)

### Preprocessing

In [3]:
# if C50 split 50-50
test_frac = 0

# if NOT C50 split 50-50
test_frac = 0.15
test_df = train_df.sample(frac=test_frac, random_state=random_seed)
train_df = train_df.drop(test_df.index).reset_index()
test_df = test_df.reset_index()

le = preprocessing.LabelEncoder()
le.fit(train_df['author'])
train_df['author_id'] = le.transform(train_df['author']).astype(int)
test_df['author_id'] = le.transform(test_df['author']).astype(int)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [4]:
print('The average length of text of an article in training set is', np.mean([len(content.split()) for content in train_df['content'].values]), 
      'and in test set is', np.mean([len(content.split()) for content in test_df['content'].values]))

The average length of text of an article in training set is 365.8708235294118 and in test set is 381.47866666666664


In [5]:
valid_frac = 0.1 / (1 - test_frac)
valid_df = train_df.sample(frac=valid_frac, random_state=random_seed)
train_df = train_df.drop(valid_df.index).reset_index()
valid_df = valid_df.reset_index()

In [6]:
def debug():
    # text = tokenizer("Commonwealth _ of Australia (_) managing", padding='max_length', max_length = 32, truncation=True, return_tensors="pt")
    # print(text)
    # mask_val = tokenizer.vocab.get('_')
    # text['attention_mask'] = torch.where(text['input_ids'] == mask_val, 0, text['attention_mask'])
    # print(text)
    text = "Westpac Banking Corp Ltd is expected to report on Tuesday that its net profit growth was reined"
    sub = "Banking"
    i = text.find(sub)
    print(i, text[i-1], text[i+len(sub)])
    if i == -1 or (i > 0 and text[i-1].isalpha()) or (i+len(sub) < len(text) and text[i+len(sub)].isalpha()):
        print('no')
    else:
        print('yes')

debug()

8    
yes


In [7]:
def mask_contents(threshold):
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), lowercase=False) #, stop_words='english')
    vectorizer.fit(pd.concat([train_df['content'], valid_df['content']]))
    X_train = vectorizer.transform(train_df['content'])
    X_valid = vectorizer.transform(valid_df['content'])
    X_test = vectorizer.transform(test_df['content'])
    feat_names = vectorizer.get_feature_names_out()
    feat_idf = vectorizer.idf_

    def mask_content(X, df):
        mask_words = set(feat_names[feat_idf >= threshold])
        ret = []
        for i in range(df['content'].shape[0]):
            text = df['content'][i]
            text = ' '.join(['_' if word in mask_words else word for word in text.split()])
          # for mask_word in mask_words:
          #   while True:
          #     i = text.find(mask_word)
          #     if i == -1 or (i > 0 and text[i-1].isalpha()) or (i+len(mask_word) < len(text) and text[i+len(mask_word)].isalpha()):
          #       break
          #     text = text[:i] + '_' + text[i+len(mask_word):]
            ret.append(text)
        return ret

    train_df['masked_content'] = mask_content(X_train, train_df)
    valid_df['masked_content'] = mask_content(X_valid, valid_df)
    test_df['masked_content'] = mask_content(X_test, test_df)

### Dataset Class

In [8]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = df['author_id'].values
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['masked_content']]
        # add attention mask
        mask_val = tokenizer.vocab.get('_')
        for text in self.texts:
            text['attention_mask'] = torch.where(text['input_ids'] == mask_val, 0, text['attention_mask'])

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

### Model Class

In [9]:
from torch import nn
from transformers import DistilBertForSequenceClassification, BertForSequenceClassification, DebertaForSequenceClassification, get_linear_schedule_with_warmup

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super().__init__()

        self.bert = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_))
        # self.dropout = nn.Dropout(dropout)
        # self.linear = nn.Linear(768, 50)
        # self.relu = nn.ReLU()

    def forward(self, input_id, labels=None):

        # pooled_output, = self.bert(input_ids=input_id, attention_mask=mask, return_dict=False)
        # pooled_output = pooled_output[:,0,:]
        # dropout_output = self.dropout(pooled_output)
        # linear_output = self.linear(dropout_output)
        # final_layer = self.relu(linear_output)
        # print(input_id.shape, labels.shape)
        final_layer = self.bert(input_id, labels=labels)

        return final_layer

### Training

In [11]:
# !git clone https://gist.github.com/NTT123/4596e5533e573c8ceab2f319ab5d36a2 jslog
# import random
# import math
# import time
# from jslog.jslogger import JSLogger

# logger = JSLogger('train/valid loss', ['train', 'valid'])
# logger_ = JSLogger('train/valid accuracy', ['train', 'valid'])

In [10]:
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

def train(model, train_data, val_data, learning_rate, epochs, batch_size):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader) * epochs,
    )

    if use_cuda:
        
        print('cuda version', torch.__version__)
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):
        
        train_loss = 0
        train_pred = []
        train_truth = []
        model.train()

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            # mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, train_label)
                
            # batch_loss = criterion(output, train_label.long())
            batch_loss = output.loss
            train_loss += batch_loss.item()
            train_pred.append(output.logits.argmax(dim=1))
            train_truth.append(train_label)

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
            lr_scheduler.step()
                
        train_truth = torch.cat(train_truth).detach().cpu().numpy()
        train_pred = torch.cat(train_pred).detach().cpu().numpy()
        train_acc = accuracy_score(train_truth, train_pred)
        train_f1 = f1_score(train_truth, train_pred, average='macro')
            
        val_loss = 0
        val_pred = []
        val_truth = []
        model.eval()

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.to(device)
                # mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, val_label)

                # batch_loss = criterion(output, val_label.long())
                batch_loss = output.loss
                val_loss += batch_loss.item()
                val_pred.append(output.logits.argmax(dim=1))
                val_truth.append(val_label)
            
        val_truth = torch.cat(val_truth).detach().cpu().numpy()
        val_pred = torch.cat(val_pred).detach().cpu().numpy()
        val_acc = accuracy_score(val_truth, val_pred)
        val_f1 = f1_score(val_truth, val_pred, average='macro')
            
        print('Epochs: {} | Train Loss: {:.4f} | Train Accuracy: {:.4f} | Train F1: {:.4f}'\
                .format(epoch_num + 1, train_loss, train_acc, train_f1))
        print('           | Val Loss:   {:.4f} | Val Accuracy:   {:.4f} | Val F1:   {:.4f}'\
                .format(val_loss, val_acc, val_f1))
            
        # logger.log(epoch_num + 1, {'train': total_loss_train / len(train_data), 'valid': total_loss_val / len(val_data)})
        # logger_.log(epoch_num + 1, {'train': total_acc_train / len(train_data), 'valid': total_acc_val / len(val_data)})
    return train_acc, train_f1, val_acc, val_f1

# logger.show()
# logger_.show()
pd.options.display.max_colwidth = 100       
EPOCHS = 10
LR = 2e-5
BATCH_SIZE = 16
model = BertClassifier()
print('There are', sum(p.numel() for p in model.parameters()), 'parameters')

mask_contents(threshold=10.)
print(train_df['masked_content'])
              
# train(model, train_df, valid_df, LR, EPOCHS, BATCH_SIZE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

There are 109489930 parameters
0       released a statement via email in which she said the decision is a clear demonstration of the im...
1       Former and current congressional candidate believes Lt. Governor gun control ballot initiative w...
2       On was suspended for after bringing an empty . shell casing to school to show his friends. [’s s...
3       presidential hopeful used to release a new ad warning that politicians are incrementally weakeni...
4       The ’s and , held in , , produced a “economic impact. ”[According to The this made the conventio...
                                                       ...                                                 
3745    In a preview clip of an interview set to air on broadcast of “,” President said Presidents and w...
3746    at the League Conference, Director said law enforcement “officers and deputies and agents” need ...
3747    at a press gathering on the campaign trail in , , Sen. ( ) responded to the controversy involvin.

### Evaluate

In [11]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    test_pred = []
    test_truth = []
    model.eval()
    
    with torch.no_grad():

        for test_input, test_label in test_dataloader:
            
            test_label = test_label.to(device)
            # mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, test_label)

            test_pred.append(output.logits.argmax(dim=1))
            test_truth.append(test_label)
        
        test_truth = torch.cat(test_truth).detach().cpu().numpy()
        test_pred = torch.cat(test_pred).detach().cpu().numpy()
        test_acc = accuracy_score(test_truth, test_pred)
        test_f1 = f1_score(test_truth, test_pred, average='macro')
    
    print('Test Accuracy:  {:.4f} | Test F1:  {:.4f}'.format(test_acc, test_f1))
    return test_acc, test_f1
    
# print(test_df['masked_content'])
# evaluate(model, test_df)

### Fine-Tuning

In [14]:
def fine_tuning(lr_list, epoch_list, batch_list, checkpoint_path='tuning2'):
    
    index_list, index2_list, index3_list = [], [], []
    train_accs, train_f1s = [], []
    test_accs, test_f1s = [], []
    
    for lr in lr_list:
        for epoch in epoch_list:
            for batch_size in batch_list:
            
                model = BertClassifier()
                # mask_contents(threshold=threshold)
                print('lr at {}, epoch at {}, batch_size at {}'.format(lr, epoch, batch_size))
                train_acc, train_f1, val_acc, val_f1 = train(model, train_df, valid_df, lr, epoch, batch_size)
                test_acc, test_f1 = evaluate(model, test_df)

                train_accs.append(train_acc)
                train_f1s.append(train_f1)
                test_accs.append(test_acc)
                test_f1s.append(test_f1)
                index_list.append(lr)
                index2_list.append(epoch)
                index3_list.append(batch_size)

                result_df = pd.DataFrame(index_list, columns=['lr'])
                result_df['epoch'] = index2_list
                result_df['batch_size'] = index3_list
                result_df['train_accs'] = train_accs
                result_df['train_f1s'] = train_f1s
                result_df['test_accs'] = test_accs
                result_df['test_f1s'] = test_f1s
                result_df['model_name'] = 'bert_uncased'
                result_df.to_csv(checkpoint_path + '.csv', index=False)
                # break
    return index_list, index2_list, index3_list, train_accs, train_f1s, test_accs, test_f1s

In [15]:
# C50 [5e-5 + 10 + 16, 2e-5 + 15 + 16, 1e-5 + 15 + 8]
# All The News []
lr_list = [2e-5] #[5e-5, 2e-5, 1e-5]
epoch_list = [15] #[10, 15]
batch_list = [8, 16, 32] #[8, 16, 32]
fine_tuning(lr_list=lr_list, epoch_list=epoch_list, batch_list=batch_list)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

lr at 2e-05, epoch at 15, batch_size at 8
cuda version 1.13.0


100%|█████████████████████████████████████████| 469/469 [01:41<00:00,  4.60it/s]


Epochs: 1 | Train Loss: 595.1981 | Train Accuracy: 0.6075 | Train F1: 0.6097
           | Val Loss:   29.6077 | Val Accuracy:   0.8580 | Val F1:   0.8653


100%|█████████████████████████████████████████| 469/469 [01:42<00:00,  4.57it/s]


Epochs: 2 | Train Loss: 177.7380 | Train Accuracy: 0.8931 | Train F1: 0.8932
           | Val Loss:   26.3167 | Val Accuracy:   0.8680 | Val F1:   0.8701


100%|█████████████████████████████████████████| 469/469 [01:42<00:00,  4.57it/s]


Epochs: 3 | Train Loss: 84.0845 | Train Accuracy: 0.9528 | Train F1: 0.9527
           | Val Loss:   16.1461 | Val Accuracy:   0.9220 | Val F1:   0.9261


 30%|████████████▌                            | 143/469 [00:31<01:11,  4.55it/s]


KeyboardInterrupt: 

### Grid Search IDF

In [14]:
import warnings

def grid_search_idf(threshold_list, checkpoint_path='checkpoint'):
    
    index_list = []
    train_accs, train_f1s = [], []
    test_accs, test_f1s = [], []
    
    for threshold in threshold_list:
        
        # warnings.filterwarnings(action='once')
        model = BertClassifier()
        mask_contents(threshold=threshold)
        print('idf removal threshold at', threshold)
        print(train_df['masked_content'])
        train_acc, train_f1, val_acc, val_f1 = train(model, train_df, valid_df, LR, EPOCHS, BATCH_SIZE)
        test_acc, test_f1 = evaluate(model, test_df)
        
        train_accs.append(train_acc)
        train_f1s.append(train_f1)
        test_accs.append(test_acc)
        test_f1s.append(test_f1)
        index_list.append(threshold)
        
        result_df = pd.DataFrame(index_list, columns=['idf_threshold'])
        result_df['train_accs'] = train_accs
        result_df['train_f1s'] = train_f1s
        result_df['test_accs'] = test_accs
        result_df['test_f1s'] = test_f1s
        result_df['model_name'] = 'bert_uncased'
        result_df.to_csv(checkpoint_path + '.csv', index=False)
        # break
    return index_list, train_accs, train_f1s, test_accs, test_f1s

In [15]:
pd.options.display.max_colwidth = 100            
EPOCHS = 15
LR = 2e-5
BATCH_SIZE = 16
grid_search_idf(threshold_list=np.arange(10., 1., -0.5))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

idf removal threshold at 10.0
0       The capital market, roundly criticised for a lack of transparency, needs an independent regulato...
1       beat the Republic 3-0 in a ice hockey game on , setting up a showdown for group supremacy agains...
2       The fall in the trade deficit to crowns in from in buoyed market sentiment, and the goods import...
3       paper concern a.s. on said its net profit fell as a shutdown of its main paper mill, coupled wit...
4       shares rallied on following the coalition government's win in elections, but analysts said the r...
                                                       ...                                                 
2245    "" -- counter-culture anthem of the 's -- is being used in a commericial for a bank and some fan...
2246    , the flamboyant stock picker hired recently to spearhead drive to dominate the market, has been...
2247    , 's biggest city and financial capital, is bracing for a near shutdown when protesters hit the ..

100%|█████████████████████████████████████████| 141/141 [01:38<00:00,  1.43it/s]


Epochs: 1 | Train Loss: 550.2232 | Train Accuracy: 0.0351 | Train F1: 0.0176
           | Val Loss:   59.8704 | Val Accuracy:   0.0680 | Val F1:   0.0356


100%|█████████████████████████████████████████| 141/141 [01:23<00:00,  1.69it/s]


...

100%|█████████████████████████████████████████| 141/141 [01:25<00:00,  1.64it/s]


Epochs: 14 | Train Loss: 72.6325 | Train Accuracy: 0.9809 | Train F1: 0.9807
           | Val Loss:   20.8237 | Val Accuracy:   0.6640 | Val F1:   0.6322


100%|█████████████████████████████████████████| 141/141 [01:26<00:00,  1.63it/s]


Epochs: 15 | Train Loss: 70.2369 | Train Accuracy: 0.9804 | Train F1: 0.9803
           | Val Loss:   20.6775 | Val Accuracy:   0.6680 | Val F1:   0.6385
Test Accuracy:  0.5080 | Test F1:  0.5028


([10.0,
  9.5,
  9.0,
  8.5,
  8.0,
  7.5,
  7.0,
  6.5,
  6.0,
  5.5,
  5.0,
  4.5,
  4.0,
  3.5,
  3.0,
  2.5,
  2.0,
  1.5],
 [0.996,
  0.9977777777777778,
  0.9968888888888889,
  0.996,
  0.9884444444444445,
  0.996,
  0.9982222222222222,
  0.9933333333333333,
  0.9942222222222222,
  0.992,
  0.9911111111111112,
  0.9888888888888889,
  0.9928888888888889,
  0.988,
  0.9857777777777778,
  0.9875555555555555,
  0.9746666666666667,
  0.9804444444444445],
 [0.9959642800473014,
  0.9977092598261774,
  0.9969006964151248,
  0.995976240283706,
  0.9877127538679704,
  0.9958688123532511,
  0.9981964351557647,
  0.9932460689747131,
  0.994009167454589,
  0.9919352127819567,
  0.9909322250261116,
  0.9881834563664834,
  0.9929590924891909,
  0.987932655916726,
  0.9854612925034312,
  0.9876574827279331,
  0.9742435640531251,
  0.9802586740963428],
 [0.6092,
  0.628,
  0.6204,
  0.6108,
  0.6092,
  0.6056,
  0.6056,
  0.5932,
  0.61,
  0.602,
  0.5864,
  0.5684,
  0.5544,
  0.536,
  0.5328,
 

### Saliency Map

In [ ]:
def get_saliency_map(model, df):
    input_text = [tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for text in df['content']]
    input = input_text[0]
    print(input.shape)
    for param in model.parameters():
        param.requires_grad = False
    model.eval()
    # input.unsqueeze_(0)
    input.requires_grad = True
    preds = model(input)
    score, indices = torch.max(preds, 1)
    #backward pass to get gradients of score predicted class w.r.t. input image
    score.backward()
    #get max along channel axis
    slc, _ = torch.max(torch.abs(input.grad[0]), dim=0)
    #normalize to [0..1]
    slc = (slc - slc.min())/(slc.max()-slc.min())
    return slc

input_slc = get_saliency_map(model, train_df)
print(input_slc.shape)
input_slc

### SHAP

In [ ]:
import shap
import scipy as sp

def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', \
        max_length=512, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

explainer = shap.Explainer(f, tokenizer)
shap_values = explainer(train_df['masked_content'][:3], fixed_context=1, batch_size=32)

In [ ]:
shap.plots.bar(shap_values.abs.mean(0)) # default

In [ ]:
shap.plots.bar(shap_values.abs.sum(0))

In [ ]:
shap.plots.bar(shap_values.abs.max(0))

In [ ]:
shap.plots.text(shap_values)